In [ ]:
#| default_exp jupyter

# Jupyter compatibility

> Use FastHTML in Jupyter notebooks
- skip_exec: true

In [ ]:
#| export
import asyncio, socket, time, uvicorn
from threading import Thread
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.common import show as _show
from fastcore.parallel import startthread
try: from IPython.display import HTML,Markdown,display
except ImportError: pass

In [ ]:
from httpx import get, AsyncClient

## Helper functions

In [ ]:
#| export
def nb_serve(app, log_level="error", port=8000, host='0.0.0.0', **kwargs):
    "Start a Jupyter compatible uvicorn server with ASGI `app` on `port` with `log_level`"
    server = uvicorn.Server(uvicorn.Config(app, log_level=log_level, host=host, port=port, **kwargs))
    async def async_run_server(server): await server.serve()
    @startthread
    def run_server(): asyncio.run(async_run_server(server))
    while not server.started: time.sleep(0.01)
    return server

In [ ]:
#| export
async def nb_serve_async(app, log_level="error", port=8000, host='0.0.0.0', **kwargs):
    "Async version of `nb_serve`"
    server = uvicorn.Server(uvicorn.Config(app, log_level=log_level, host=host, port=port, **kwargs))
    asyncio.get_running_loop().create_task(server.serve())
    while not server.started: await asyncio.sleep(0.01)
    return server

In [ ]:
#| export
def is_port_free(port, host='localhost'):
    "Check if `port` is free on `host`"
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        sock.bind((host, port))
        return True
    except OSError: return False
    finally: sock.close()

In [ ]:
#| export
def wait_port_free(port, host='localhost', max_wait=3):
    "Wait for `port` to be free on `host`"
    start_time = time.time()
    while not is_port_free(port):
        if time.time() - start_time>max_wait: return print(f"Timeout")
        time.sleep(0.1)

## Using FastHTML in Jupyter

In [ ]:
#| export
def show(*s):
    "Same as fasthtml.components.show, but also adds `htmx.process()`"
    if IN_NOTEBOOK: return _show(*s, Script('htmx.process(document.body)'))
    return _show(*s)

@patch
def _repr_markdown_(self:FT): return to_xml(Div(self, Script('if (window.htmx) htmx.process(document.body)')))

In [ ]:
#| export
def htmx_config_port(port=8000):
    display(HTML('''
<script>
document.body.addEventListener('htmx:configRequest', (event) => {
    if(event.detail.path.includes('://')) return;
    htmx.config.selfRequestsOnly=false;
    event.detail.path = `${location.protocol}//${location.hostname}:%s${event.detail.path}`;
});
</script>''' % port))

In [ ]:
#| export
class JupyUvi:
    "Start and stop a Jupyter compatible uvicorn server with ASGI `app` on `port` with `log_level`"
    def __init__(self, app, log_level="error", host='0.0.0.0', port=8000, start=True, **kwargs):
        self.kwargs = kwargs
        store_attr(but='start')
        self.server = None
        if start: self.start()
        htmx_config_port(port)

    def start(self):
        self.server = nb_serve(self.app, log_level=self.log_level, host=self.host, port=self.port, **self.kwargs)

    def stop(self):
        self.server.should_exit = True
        wait_port_free(self.port)

Creating an object of this class also starts the Uvicorn server. It runs in a separate thread, so you can use normal HTTP client functions in a notebook. 

In [ ]:
app = FastHTML()
rt = app.route

@app.route
def index(): return 'hi'

port = 8000
server = JupyUvi(app, port=port)

In [ ]:
get(f'http://localhost:{port}').text

'hi'

You can stop the server, modify routes, and start the server again without restarting the notebook or recreating the server or application.

### Using a notebook as a web app

You can also run an HTMX web app directly in a notebook. You might find it convenient to use *auto_id* mode, in which the ID of an `FT` object is automatically generated if not provided.

In [ ]:
fh_cfg['auto_id' ]=True

After importing `fasthtml.jupyter`, FT components render directly in the notebook.

In [ ]:
(c := Div('Cogito ergo sum'))

<div id="_TKB8uH2dQyuysr7uMoHg1w">
  <div id="_OuGkA_SAR62XC1cBsUPAwA">Cogito ergo sum</div>
<script id="_O7x83b64QoCKuLp6V6Qxkw">if (window.htmx) htmx.process(document.body)</script></div>


Handlers are written just like a regular web app:

In [ ]:
@rt
def hoho(): return P('loaded!'), Div('hee hee', id=c, hx_swap_oob='true')

All the usual `hx_*` attributes can be used:

In [ ]:
P('not loaded', hx_get=hoho, hx_trigger='load')

<div id="_Ea_nzHJeQG_3hj68pbJ_mQ">
  <p hx-get="/hoho" hx-trigger="load" id="_2BPhAdYwTAiA1lLrU-7-5w">not loaded</p>
<script id="_Y_kyC8_oTdCB4gFQ4Rc3Qw">if (window.htmx) htmx.process(document.body)</script></div>


FT components can be used directly both as `id` values and as `hx_target` values.

In [ ]:
(c := Div(''))

<div id="_Oj-JCsNBS0yOX8yZ_PvRmg">
  <div id="_37uuVSvXQ-a72m-bWqL9Ow"></div>
<script id="_VEF-wKPfSp2u_apvxJxmWA">if (window.htmx) htmx.process(document.body)</script></div>


In [ ]:
@rt
def foo(): return Div('foo bar')
P('hi', hx_get=foo, hx_trigger='load', hx_target=c)

<div id="_CzYHFVtdTCur3ZnHkxmU3g">
  <p hx-get="/foo" hx-trigger="load" hx-target="#_37uuVSvXQ-a72m-bWqL9Ow" id="_VnfiEFZiQcavFO3IdDfe7A">hi</p>
<script id="_AAmzR0FFQV_6MEepI-T0Ig">if (window.htmx) htmx.process(document.body)</script></div>


In [ ]:
server.stop()

### Running apps in an IFrame

Using an IFrame can be a good idea to get complete isolation of the styles and scripts in an app. The `HTMX` function creates an auto-sizing IFrame for a web app.

In [ ]:
#| export
def HTMX(path="", host='localhost', port=8000, iframe_height="auto"):
    "An iframe which displays the HTMX application in a notebook."
    return HTML(f'<iframe src="http://{host}:{port}{str(path)}" style="width: 100%; height: {iframe_height}; border: none;" ' + """onload="{
        let frame = this;
        window.addEventListener('message', function(e) {
            if (e.source !== frame.contentWindow) return; // Only proceed if the message is from this iframe
            if (e.data.height) frame.style.height = (e.data.height+1) + 'px';
        }, false);
    }" allow="accelerometer; autoplay; camera; clipboard-read; clipboard-write; display-capture; encrypted-media; fullscreen; gamepad; geolocation; gyroscope; hid; identity-credentials-get; idle-detection; magnetometer; microphone; midi; payment; picture-in-picture; publickey-credentials-get; screen-wake-lock; serial; usb; web-share; xr-spatial-tracking"></iframe> """)

In [ ]:
@rt
def index():
    return Div(
        P(A('Click me', hx_get=update, hx_target='#result')),
        P(A('No me!', hx_get=update, hx_target='#result')),
        Div(id='result'))

@rt
def update(): return Div(P('Hi!'),P('There!'))

In [ ]:
server.start()

In [ ]:
# Run the notebook locally to see the HTMX iframe in action
# HTMX()

In [ ]:
server.stop()

In [ ]:
#| export
def ws_client(app, nm='', host='localhost', port=8000, ws_connect='/ws', frame=True, link=True, **kwargs):
    path = f'/{nm}'
    c = Main('', cls="container", id=unqid())
    @app.get(path)
    def f():
        return Div(c, id=nm or '_dest', hx_trigger='load',
                   hx_ext="ws", ws_connect=ws_connect, **kwargs)
    if link: display(HTML(f'<a href="http://{host}:{port}{path}" target="_blank">open in browser</a>'))
    if frame: display(HTMX(path, host=host, port=port))
    def send(o): asyncio.create_task(app._send(o))
    c.on(send)
    return c

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()